In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [12]:
df=pd.read_csv("/kaggle/input/126-years-of-historical-olympic-dataset/Olympic_Medal_Tally_History.csv")

In [13]:
df

,edition,edition_id,year,country,country_noc,gold,silver,bronze,total
0,1896 Summer Olympics,1,1896,United States,USA,11,7,2,20
1,1896 Summer Olympics,1,1896,Greece,GRE,10,18,19,47
2,1896 Summer Olympics,1,1896,Germany,GER,6,5,2,13
3,1896 Summer Olympics,1,1896,France,FRA,5,4,2,11
4,1896 Summer Olympics,1,1896,Great Britain,GBR,2,3,2,7
...,...,...,...,...,...,...,...,...,...
1802,1906 Intercalated,4,1906,Canada,CAN,1,1,0,2
1803,1906 Intercalated,4,1906,Norway,NOR,1,1,0,2
1804,1906 Intercalated,4,1906,Netherlands,NED,0,1,2,3
1805,1906 Intercalated,4,1906,Australia,AUS,0,0,3,3


In [14]:
df.sample()

,edition,edition_id,year,country,country_noc,gold,silver,bronze,total
539,1976 Summer Olympics,19,1976,Jamaica,JAM,1,1,0,2


In [15]:
df.isnull().sum()

edition        0
edition_id     0
year           0
country        0
country_noc    0
gold           0
silver         0
bronze         0
total          0
dtype: int64

In [27]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["edition","country","country_noc"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())
one["edition_id"]=df["edition_id"]

In [28]:
one

,edition_1896 Summer Olympics,edition_1900 Summer Olympics,edition_1904 Summer Olympics,edition_1906 Intercalated,edition_1908 Summer Olympics,edition_1912 Summer Olympics,edition_1920 Summer Olympics,edition_1924 Summer Olympics,edition_1924 Winter Olympics,edition_1928 Summer Olympics,...,country_noc_URU,country_noc_USA,country_noc_UZB,country_noc_VEN,country_noc_VIE,country_noc_WIF,country_noc_YUG,country_noc_ZAM,country_noc_ZIM,edition_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1803,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1804,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [29]:
Y=df[["gold","silver","bronze","total"]]

In [36]:
Y.nunique().sum()

234

In [31]:
X_train,X_test,Y_train,Y_test=train_test_split(one,Y,test_size=0.2,random_state=42)

In [32]:
random_classif=RandomForestClassifier()
multi_output_classif=MultiOutputClassifier(random_classif)
multi_output_classif.fit(X_train,Y_train)
multi_output_classif.score(X_test,Y_test)

0.04143646408839779

# # Deep learning

In [40]:
from keras.layers import Dense,Input,BatchNormalization,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [41]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [42]:
X_train.shape

(1445, 364)

In [47]:
inputs=Input(shape=(364,))
D=Dense(32,activation='relu',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dense(32,activation='relu',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(D)
D=Dropout(0.5)(B)
D=Dense(64,activation='relu',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
D=Dense(64,activation='relu',kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
B=BatchNormalization()(D)
outputs=Dense(4,activation="softmax")(B)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [48]:
model.fit(X_train,Y_train,batch_size=32,epochs=50,validation_data=(X_test,Y_test),callbacks=[early_stopping])

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2627 - loss: 46.4038 - val_accuracy: 0.2652 - val_loss: 38.3628
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3333 - loss: 40.9535 - val_accuracy: 0.3536 - val_loss: 35.1897
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3705 - loss: 45.4951 - val_accuracy: 0.4586 - val_loss: 35.6416
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4477 - loss: 47.9241 - val_accuracy: 0.6878 - val_loss: 47.0008
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4673 - loss: 77.7104 - val_accuracy: 0.2044 - val_loss: 59.9102
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4814 - loss: 104.8034 - val_accuracy: 0.1740 - val_loss: 84.4377
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5017 - loss: 135.0323 - val_accuracy: 0.1934 - val_loss: 88.1826
Epoch 7: early stopping


In [49]:
loss,accuracy=model.evaluate(X_test,Y_test)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2038 - loss: 98.6568  
